## Setup

In [ ]:
import os

# importing scripting dependencies
import boa
from eth_account import Account

In [ ]:
from dotenv import load_dotenv

load_dotenv()
PRIVATE_KEY = os.environ.get("WEB3_TESTNET_PK")
# RPC_URL = "https://sepolia.optimism.io"
RPC_URL = "https://sepolia.base.org"
# RPC_URL = "https://mainnet.optimism.io"

## Select mode
Production mode cell is intentionally executed first to avoid deploying things by accident. If one runs all the cells sequentially things will be executed in forking mode.

To deploy in production mode, manually skip the execution of the fork mode cell.

In [ ]:
# production mode (to deploy on ethereum)
deployer = Account.from_key(PRIVATE_KEY)
eth_env = boa.set_network_env(RPC_URL)
# boa.set_env(eth_env)

# this automatically sets the eoa as the deployer
boa.env.add_account(deployer)
print(f"Deploying with {deployer.address}")

# I. Deployment

### 2. LZContract

In [ ]:
# II. Then deploy the DepositLimitModule
contract_deployer = boa.load_partial("../contracts/LZContract.vy")
LZ_ENDPOINT_BASE_SEPOLIA = "0x6EDCE65403992e310A62460808c4b910D972f10f"
LZ_ENDPOINT_ID = 40245

contract = contract_deployer(
    LZ_ENDPOINT_BASE_SEPOLIA,  # endpoint on sepolia base
)

print(f"Deposit limit module deployed at {contract.address}")

# II. Post-deployment interactions

In [ ]:
from web3 import Web3
import os
import logging
import subprocess

LZ_ENDPOINT_ID = 40245


def get_vyper_abi():
    command = ["vyper", "../contracts/LZContract.vy", "-f", "abi_python"]
    try:
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"Error: {e.stderr}"


logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")
# RPC endpoints

CONTRACT_ADDRESS = contract.address

try:
    pk = os.environ["WEB3_TESTNET_PK"]
except KeyError:
    raise EnvironmentError("Please set WEB3_TESTNET_PK environment variable")
# execute vyper contracts/LZContract.vy -f abi_python in cli and get the output
ABI = get_vyper_abi()

w3 = Web3(Web3.HTTPProvider("https://sepolia.base.org"))
contract_w3 = w3.eth.contract(address=CONTRACT_ADDRESS, abi=ABI)
account = w3.eth.account.from_key(pk)
gas_limit = 500_000
fee = contract_w3.functions.quote_fee(LZ_ENDPOINT_ID, CONTRACT_ADDRESS, _gas_limit=gas_limit).call()
logging.info(f"Fee: {fee}")


func = contract_w3.functions.send_message(
    _dstEid=LZ_ENDPOINT_ID, _receiver=CONTRACT_ADDRESS, _gas_limit=gas_limit
)
tx = func.build_transaction(
    {
        "from": account.address,
        "nonce": w3.eth.get_transaction_count(account.address),
        "value": fee,
    }
)
tx["gas"] = int(5 * w3.eth.estimate_gas(tx))
signed_tx = w3.eth.account.sign_transaction(tx, private_key=account.key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
logging.info(f"Tx: {tx_hash.hex()}")